In [96]:
import numpy as np
from main import *
from copy import deepcopy
from random import choice

In [97]:
def rule1(sp, **kwargs):
    """maximize the distance from the head of task to the depot"""
    return max(kwargs["tasks"], key=lambda x: sp[kwargs["depot"], x[0][0]])

def rule2(sp, **kwargs):
    """minimize the distance from the head of task to the depot"""
    return min(kwargs["tasks"], key=lambda x: sp[kwargs["depot"], x[0][0]])

def rule3(sp, **kwargs):
    """maximize the term dem(t)/sc(t)"""
    return max(kwargs["tasks"], key=lambda x: x[1] / sp[x[0]])

def rule4(sp, **kwargs):
    """minimize the term dem(t)/sc(t)"""
    return min(kwargs["tasks"], key=lambda x: x[1] / sp[x[0]])

def rule5(sp, **kwargs):
    """use rule1 if the vehicle is less than half full, otherwise use rule2"""
    return rule1(sp, **kwargs) if kwargs["remain_cap"] / kwargs["capacity"] > 0.5 else rule2(sp, **kwargs)

num = 5

In [98]:
class Vehicle:
    def __init__(self, remain_cap, route):
        self.remain_cap = remain_cap
        self.route = route

    def __str__(self):
        return str(self.remain_cap) + "\n" + str(self.route)
    

In [99]:
def generate_solu_by_rule(sp, tasks, depot, capacity, num_vehicles, rule):
    dummy_task = ((depot, depot), 0)
    solu = []
    
    # 多辆车
    for _ in range(num_vehicles):
        if len(tasks) == 0:
            break

        remain_cap = capacity
        cur_vertex = depot
        route = [dummy_task]

        while len(tasks) > 0:
            valid_tasks = []
            for task in tasks:
                if task[1] <= remain_cap:
                    valid_tasks.append(task)

            if len(valid_tasks) == 0:
                # 没有有效的任务
                break

            elif len(valid_tasks) == 2:
                # 因为一个tasks有两条记录
                # 只有一个有效任务，最短路径过去
                task = valid_tasks[0] if valid_tasks[0][0][0] < valid_tasks[1][0][0] else valid_tasks[1]

                # 添加路线
                route.append(task)

                # ! 修改车的位置，是tail，不是head
                cur_vertex = task[0][1]  

                # 移除task
                tasks.remove(valid_tasks[0])
                tasks.remove(valid_tasks[1])

                # 减少容量
                remain_cap -= task[1]

                # 一定是最后一个了
                break

            else:
                # 有多个任务满足条件，选一个距离最短的
                min_dist = min(
                    valid_tasks, key=lambda x: sp[cur_vertex, x[0][0]])[1]
                # 检查是否有多个距离最短的点
                min_dist_tasks = [t for t in valid_tasks if t[1] == min_dist]
                if len(min_dist_tasks) == 1:  # 1，只有一个
                    task = min_dist_tasks[0]
                else:  # 多个，使用rule
                    # !!!!!!! tasks = min_dist_tasks，不是tasks !!!!!!!!!!!
                    task = rule(sp, 
                                depot=depot, 
                                tasks=min_dist_tasks, 
                                capacity=capacity, 
                                remain_cap=remain_cap)
                
                route.append(task)
                cur_vertex = task[0][1]  # ! 修改车的位置，是tail，不是head
                remain_cap -= task[1] 
                tasks.remove(task)
                tasks.remove(((task[0][1], task[0][0]), task[1]))
        
        route.append(dummy_task)
        solu.append(Vehicle(remain_cap, route))
    
    return solu

In [100]:
def generate_solu_random(tasks, depot, capacity, num_vehicles):
    dummy_task = ((depot, depot), 0)
    solu = []
    
    # 多辆车
    for _ in range(num_vehicles):
        if len(tasks) == 0:
            break

        remain_cap = capacity
        route = [dummy_task]

        while len(tasks) > 0:
            valid_tasks = []
            for task in tasks:
                if task[1] <= remain_cap:
                    valid_tasks.append(task)

            if len(valid_tasks) == 0:
                # 没有有效的任务
                break

            else:
                task = choice(valid_tasks)
                
                route.append(task)
                remain_cap -= task[1]
                tasks.remove(task)
                tasks.remove(((task[0][1], task[0][0]), task[1]))
        
        # 一辆车回到终点后，加入dummy_task
        route.append(dummy_task)
        solu.append(Vehicle(remain_cap, route))
    
    return solu

In [101]:
def generate_solus(sp, tasks, depot, capacity, num_vehicles, num_routes):
    return [generate_solu_by_rule(
        sp, deepcopy(tasks),
        depot, capacity, num_vehicles, rule)
        for rule in [rule1, rule2, rule3, rule4, rule5]] + \
        [generate_solu_random(deepcopy(tasks), depot, 
                               capacity, num_vehicles) for _ in range(num_routes - 5)]


In [104]:
info = read_file("sample2.dat")
sp = shortest_path(info["graph"])
depot, capacity, tasks = info["depot"], info["capacity"], info["tasks"]
num_vehicles = info["num_vehicles"]

In [106]:
solutions = generate_solus(sp, tasks, depot, capacity, num_vehicles, 20)
for solu in solutions:
    print()
    for vehicle in solu:
        print(vehicle)
    


0
[((0, 0), 0), ((18, 19), 32), ((19, 75), 8), ((11, 10), 32), ((42, 43), 78), ((44, 43), 12), ((12, 15), 13), ((15, 11), 29), ((56, 57), 78), ((58, 68), 11), ((50, 48), 10), ((20, 50), 2), ((0, 0), 0)]
0
[((0, 0), 0), ((57, 68), 32), ((68, 3), 75), ((22, 74), 17), ((74, 21), 24), ((21, 20), 17), ((14, 17), 38), ((18, 17), 41), ((16, 14), 26), ((13, 12), 7), ((9, 8), 20), ((49, 48), 8), ((0, 0), 0)]
2
[((0, 0), 0), ((1, 0), 32), ((41, 56), 14), ((34, 40), 15), ((31, 34), 86), ((54, 55), 6), ((46, 45), 33), ((45, 43), 13), ((43, 58), 28), ((57, 58), 25), ((59, 57), 33), ((65, 61), 16), ((51, 49), 2), ((0, 0), 0)]
1
[((0, 0), 0), ((2, 1), 14), ((3, 1), 17), ((4, 3), 56), ((10, 58), 78), ((60, 59), 39), ((61, 59), 23), ((32, 31), 30), ((33, 31), 43), ((51, 53), 4), ((0, 0), 0)]
54
[((0, 0), 0), ((47, 46), 10), ((48, 46), 9), ((18, 20), 38), ((30, 22), 40), ((31, 30), 58), ((62, 61), 30), ((67, 65), 45), ((62, 64), 21), ((0, 0), 0)]

0
[((0, 0), 0), ((0, 1), 32), ((1, 2), 14), ((1, 3), 17